In [ ]:
!pip install snowflake-snowpark-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.7/419.7 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: platformdirs
    Found existing installation: platformdirs 4.2.0
    Uninstalling platformdirs-4.2.0:
      Successfully uninstalled platformdirs-4.2.0


In [ ]:
from snowflake.snowpark import Session
from snowflake.snowpark import functions as F
from snowflake.snowpark.functions import year, month
from snowflake.snowpark.functions import col, avg, stddev_pop, corr
import numpy as np
import pandas as pd
from snowflake.snowpark.exceptions import SnowparkSQLException
import statsmodels.api as sm

In [ ]:
parameters = {
...    "account": "YEZEPEO-DATASCIENCE",
...    "user": "VNSHARED",
...    "password": "Columbia2024!",
...    "role": "SYSADMIN", # optional
...    "warehouse": "ANALYSIS", # optional
...   "database": "KRIS_DATA", # optional
...   "schema":"PUBLIC",  # optional
...  }

In [ ]:
session= Session.builder.configs(parameters).create()

# Urban Highway Passage (Santiago)

Full dataset - no p-values

In [ ]:
df_v = session.sql('select * from CONSOLIDATED_COPY')
df_v = df_v.select([F.col(col_name).alias(col_name.replace('"', '')) for col_name in df_v.columns])
df_v = df_v.select("FECHAEMISION", "DIASCOMPENSACION")

In [ ]:
df_v.columns

['FECHAEMISION', 'DIASCOMPENSACION']

In [ ]:
# Step 1: Extract year and month, then identify unique year-month combinations
df_v_with_year_month = df_v.with_column("YEAR", year(col("FECHAEMISION")))\
                          .with_column("MONTH", month(col("FECHAEMISION")))
unique_year_months = df_v_with_year_month.select("YEAR", "MONTH").distinct().collect()

# Step 2: Iterate and filter for each year-month combination
filtered_dfs = []
for ym_row in unique_year_months:
    year_value, month_value = ym_row["YEAR"], ym_row["MONTH"]
    df_filtered = df_v_with_year_month.filter((col("YEAR") == year_value) & (col("MONTH") == month_value))
    filtered_dfs.append(df_filtered)

# Step 3: Union results
final_df = filtered_dfs[0]
for df in filtered_dfs[1:]:
    final_df = final_df.union(df)

In [ ]:
df_highway = session.sql('select * from TRAFFIC_SANTIAGO_COPY')

In [ ]:
df_highway = df_highway.select([F.col(col_name).alias(col_name.replace('"', '')) for col_name in df_highway.columns])

In [ ]:
#print(df_highway.columns[2])

"Descripción series"


In [ ]:
df_highway.columns

['EXTRACTIONDATE', 'REG', '"Descripción series"', 'YEAR', 'DATE', 'INDEX']

In [ ]:
filtered_df_highway = df_highway.filter(df_highway['Descripción series'] == "Pasada de vehículos por pórticos de autopistas urbanas, unidades")

In [ ]:
filtered_df_highway.show()

--------------------------------------------------------------------------------------------------------------------------
|"EXTRACTIONDATE"  |"REG"  |"Descripción series"                                |"YEAR"  |"DATE"      |"INDEX"           |
--------------------------------------------------------------------------------------------------------------------------
|2024-03-23        |16.0   |Pasada de vehículos por pórticos de autopistas ...  |2014.0  |2014-12-31  |99055550.25       |
|2024-03-23        |16.0   |Pasada de vehículos por pórticos de autopistas ...  |2015.0  |2015-12-31  |107355602.916667  |
|2024-03-23        |16.0   |Pasada de vehículos por pórticos de autopistas ...  |2016.0  |2016-12-31  |114798995.75      |
|2024-03-23        |16.0   |Pasada de vehículos por pórticos de autopistas ...  |2017.0  |2017-12-31  |120820262.416667  |
|2024-03-23        |16.0   |Pasada de vehículos por pórticos de autopistas ...  |2018.0  |2018-12-31  |123469393.5       |
|2024-03-23     

In [ ]:
# Extract year and month for 'FechaEmision' in df_v
df_v = df_v.withColumn('Year', year('FECHAEMISION'))
df_v = df_v.withColumn('Month', month('FECHAEMISION'))

# Extract year and month for 'DATE' in df_debt
filtered_df_highway = filtered_df_highway.withColumn('Year', year('DATE'))
filtered_df_highway = filtered_df_highway.withColumn('Month', month('DATE'))

# Join the dataframes on year and month
df_with_highway= df_v.join(filtered_df_highway, (df_v['Year'] == filtered_df_highway['YEAR']))

# Select the desired columns including 'Debt(%GDP)' from df_debt
# Add any other columns you need from df_v and df_debt
df_with_highway = df_with_highway.select(df_v['*'], filtered_df_highway['INDEX'])

In [ ]:
# Upload as table to perform linear regression.
# Define the name of the table you want to create or overwrite in Snowflake
table_name = 'table_with_highway'

# Write the DataFrame to a new table in Snowflake
df_with_highway.write.mode('overwrite').save_as_table(table_name)

In [ ]:
tb = session.sql('select * from TABLE_WITH_HIGHWAY ')
print(tb.columns)

['FECHAEMISION', 'DIASCOMPENSACION', '"l_8e7d_YEAR"', '"l_8e7d_MONTH"', 'INDEX']


In [ ]:
# Perform the linear regression
regression_query = """
SELECT
    REGR_SLOPE("DIASCOMPENSACION", "INDEX") AS slope,
    REGR_INTERCEPT("DIASCOMPENSACION", "INDEX") AS intercept,
    REGR_R2("DIASCOMPENSACION", "INDEX") AS r_squared,
    REGR_COUNT("DIASCOMPENSACION", "INDEX") AS sample_size,
    REGR_SXX("DIASCOMPENSACION", "INDEX") AS sxx,
    REGR_SYY("DIASCOMPENSACION", "INDEX") AS syy,
    REGR_SXY("DIASCOMPENSACION", "INDEX") AS sxy
FROM
    table_with_highway;
"""

# Execute the query
try:
    regression_results = session.sql(regression_query).collect()
    # Print the results
    for result in regression_results:
        print(f"Slope: {result['SLOPE']}")
        print(f"Intercept: {result['INTERCEPT']}")
        print(f"R-squared: {result['R_SQUARED']}")
        print(f"Sample size: {result['SAMPLE_SIZE']}")
        print(f"SXX: {result['SXX']}")
        print(f"SYY: {result['SYY']}")
        print(f"SXY: {result['SXY']}")
except SnowparkSQLException as e:
    print(f"An error occurred: {e}")

Slope: -1.5197081466227312e-06
Intercept: 264.83537333261603
R-squared: 0.005571132569357686
Sample size: 334035865
SXX: 5.342379472955876e+22
SYY: 22146832611918.71
SXY: -8.118857607401062e+16


limited dataset - with p-value

limit 10000 observations in each year

In [ ]:
df_v=session.sql('select * from CONSOLIDATED_COPY')

In [ ]:
df_v = df_v.select([F.col(col_name).alias(col_name.replace('"', '')) for col_name in df_v.columns])

In [ ]:
# Step 1: Identify unique years
df_v_with_year = df_v.with_column("YEAR", year(col("FECHAEMISION")))
unique_years = df_v_with_year.select("YEAR").distinct().collect()

# Step 2: Iterate and filter for each year
filtered_dfs = []
for year_row in unique_years:
    year_value = year_row["YEAR"]
    df_filtered = df_v_with_year.filter(col("YEAR") == year_value).limit(10000)
    filtered_dfs.append(df_filtered)

# Step 3: Union results
final_df = filtered_dfs[0]
for df in filtered_dfs[1:]:
    final_df = final_df.union(df)

In [ ]:
df_highway = session.sql('select * from TRAFFIC_SANTIAGO_COPY')

In [ ]:
df_highway = df_highway.select([F.col(col_name).alias(col_name.replace('"', '')) for col_name in df_highway.columns])

In [ ]:
filtered_df_highway = df_highway.filter(df_highway['Descripción series'] == "Pasada de vehículos por pórticos de autopistas urbanas, unidades")

In [ ]:
filtered_df_highway.show()

--------------------------------------------------------------------------------------------------------------------------
|"EXTRACTIONDATE"  |"REG"  |"Descripción series"                                |"YEAR"  |"DATE"      |"INDEX"           |
--------------------------------------------------------------------------------------------------------------------------
|2024-03-23        |16.0   |Pasada de vehículos por pórticos de autopistas ...  |2014.0  |2014-12-31  |99055550.25       |
|2024-03-23        |16.0   |Pasada de vehículos por pórticos de autopistas ...  |2015.0  |2015-12-31  |107355602.916667  |
|2024-03-23        |16.0   |Pasada de vehículos por pórticos de autopistas ...  |2016.0  |2016-12-31  |114798995.75      |
|2024-03-23        |16.0   |Pasada de vehículos por pórticos de autopistas ...  |2017.0  |2017-12-31  |120820262.416667  |
|2024-03-23        |16.0   |Pasada de vehículos por pórticos de autopistas ...  |2018.0  |2018-12-31  |123469393.5       |
|2024-03-23     

In [ ]:
filtered_df_highway = filtered_df_highway.to_pandas()
final_df=final_df.to_pandas()

In [ ]:
#Convert 'FechaEmision' and 'DATE' to datetime
final_df['FECHAEMISION'] = pd.to_datetime(final_df['FECHAEMISION'])
filtered_df_highway['DATE'] = pd.to_datetime(filtered_df_highway['DATE'])

# Extract the year from 'FechaEmision' and 'Year'
final_df['YEAR'] = final_df['FECHAEMISION'].dt.year
filtered_df_highway['YEAR'] = filtered_df_highway['YEAR'].astype(int)

# Merge the dataframes on the 'Year' column
df_with_highway = pd.merge(final_df, filtered_df_highway[['YEAR', 'INDEX']], on='YEAR', how='inner')

# Drop the extra 'Year' column if you don't need it
df_with_highway.drop('YEAR', axis=1, inplace=True)

# Check the result
df_with_highway

,FECHAEXTRACCION,NUMERODOCUMENTO,NUMEROFOLIO,FECHAEMISION,FECHAVENCIMIENTO,FECHAREGISTRO,BUSINESSPARTNERID,CUENTACONTRATO,CLAVERECONCILIACION,CABECERA_FECHAHORACREACION,...,TRAMOMORA,INHABILITADO,CONCESIONARIA,TIPOCLIENTE,TIPOOPERACION,TIPODOCUMENTO,TIPOPRODUCTO,STATUSCOMPENSACION,STATUSCOMPENSACIONTMP,INDEX
0,2024-01-15 13:52:32.627,5.440048e+10,9367034.0,2017-04-26,2017-05-11,2017-04-26,0012810227,001102157946,17115FA060AF,2021-04-25 00:09:47,...,Pago,Cliente Tag,AVN,Natural,Peaje,Boleta,Tag,Pagada,Pagada,1.208203e+08
1,2024-01-15 13:52:32.627,5.440048e+10,9417020.0,2017-05-05,2017-05-22,2017-05-04,0011326883,001101499953,17123FA010CH,2021-04-25 00:17:20,...,Pago,Cliente Tag,AVN,Natural,Intereses,Boleta,Tag,Pagada,Pagada,1.208203e+08
2,2024-01-15 13:52:32.627,5.440048e+10,9448562.0,2017-05-09,2017-05-24,2017-05-09,0010647113,001100402788,17128FA020AX,2021-04-25 00:21:55,...,Pago,Cliente Tag,TSC,Natural,Peaje,Boleta,Tag,Pagada,Pagada,1.208203e+08
3,2024-01-15 13:52:32.627,5.440048e+10,3848977.0,2017-05-09,2017-05-24,2017-05-09,0010510108,001100265706,17128FA020AX,2021-04-25 00:21:55,...,Pago,Cliente Tag,AVN,Juridico,Peaje,Factura Electronica,Tag,Pagada,Pagada,1.208203e+08
4,2024-01-15 13:52:32.627,5.440048e+10,3848977.0,2017-05-09,2017-05-24,2017-05-09,0010510108,001100265706,17128FA020AX,2021-04-25 00:21:55,...,Pago,Cliente Tag,AVN,Juridico,Peaje,Factura Electronica,Tag,Pagada,Pagada,1.208203e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,2024-01-15 13:52:32.627,6.000135e+11,6639230.0,2023-07-04,2023-07-19,2023-07-04,0012102059,001101548118,230704R4/002,2023-07-05 07:12:29,...,[181 a 360] Dias,Cliente Tag,AVN,Juridico,Intereses,Factura Electronica,Tag,Impaga,Impaga,1.311635e+08
99996,2024-01-15 13:52:32.627,5.395134e+10,32368880.0,2023-09-26,2023-10-11,2023-09-26,0013195952,001102493788,23268FA060BD,2023-09-27 06:13:51,...,[91 a 180] Dias,Cliente Tag,AVN,Natural,Peaje,Boleta,Tag,Impaga,Impaga,1.311635e+08
99997,2024-01-15 13:52:32.627,1.760240e+10,31236765.0,2023-05-16,2023-05-31,2023-05-17,0012850656,001102193774,23136FA040BC,2023-05-18 07:06:02,...,Pago,Cliente Tag,AVN,Natural,Peaje,Boleta,Tag,Pagada,Pagada,1.311635e+08
99998,2024-01-15 13:52:32.627,5.605118e+10,29561927.0,2023-01-04,2023-01-19,2023-01-04,0012729528,001102084043,23003FA010AD,2023-01-05 06:40:43,...,[361+] Dias,Cliente Tag,AVN,Natural,Peaje,Boleta,Tag,Impaga,Impaga,1.311635e+08


In [ ]:
df_filtered = df_with_highway.dropna(subset=['INDEX', 'DIASCOMPENSACION'])

# Calculate correlation
correlation = df_filtered['INDEX'].corr(df_filtered['DIASCOMPENSACION'])

# Use statsmodels library for detailed statistics
X = sm.add_constant(df_filtered['INDEX'])  # Adding a constant term to the predictor
y = df_filtered['DIASCOMPENSACION']

model = sm.OLS(y, X)
results = model.fit()

# Print summary
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:       DIASCOMPENSACION   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     343.8
Date:                Tue, 26 Mar 2024   Prob (F-statistic):           1.31e-76
Time:                        20:54:49   Log-Likelihood:            -6.5106e+05
No. Observations:               92651   AIC:                         1.302e+06
Df Residuals:                   92649   BIC:                         1.302e+06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        253.6165      8.689     29.188      0.0

In [ ]:
print("Model Coefficients:")
print(f"Intercept: {results.params[0]:.3f}")
print(f"Slope (DIASCOMPENSACION): {results.params[1]:.3f}")

print("\nModel P-values:")
print(f"P-value (Intercept): {results.pvalues[0]:.3f}")
print(f"P-value (DIASCOMPENSACION): {results.pvalues[1]:.3f}")

print("\nModel R-squared: {:.3f}".format(results.rsquared))
print("Model Standard Errors:")
print(f"Standard Error (Intercept): {results.bse[0]:.3f}")
print(f"Standard Error (DIASCOMPENSACION): {results.bse[1]:.3f}")

Model Coefficients:
Intercept: 253.617
Slope (DIASCOMPENSACION): -0.000

Model P-values:
P-value (Intercept): 0.000
P-value (DIASCOMPENSACION): 0.000

Model R-squared: 0.004
Model Standard Errors:
Standard Error (Intercept): 8.689
Standard Error (DIASCOMPENSACION): 0.000


# Interurban Toll Passage (Santiago)

Full dataset - no p-values

In [ ]:
df_v = session.sql('select * from CONSOLIDATED_COPY')
df_v = df_v.select([F.col(col_name).alias(col_name.replace('"', '')) for col_name in df_v.columns])
df_v = df_v.select("FECHAEMISION", "DIASCOMPENSACION")

In [ ]:
df_v.columns

['FECHAEMISION', 'DIASCOMPENSACION']

In [ ]:
# Step 1: Extract year and month, then identify unique year-month combinations
df_v_with_year_month = df_v.with_column("YEAR", year(col("FECHAEMISION")))\
                          .with_column("MONTH", month(col("FECHAEMISION")))
unique_year_months = df_v_with_year_month.select("YEAR", "MONTH").distinct().collect()

# Step 2: Iterate and filter for each year-month combination
filtered_dfs = []
for ym_row in unique_year_months:
    year_value, month_value = ym_row["YEAR"], ym_row["MONTH"]
    df_filtered = df_v_with_year_month.filter((col("YEAR") == year_value) & (col("MONTH") == month_value))
    filtered_dfs.append(df_filtered)

# Step 3: Union results
final_df = filtered_dfs[0]
for df in filtered_dfs[1:]:
    final_df = final_df.union(df)

In [ ]:
df_toll = session.sql('select * from TRAFFIC_SANTIAGO_COPY')

In [ ]:
df_toll = df_toll.select([F.col(col_name).alias(col_name.replace('"', '')) for col_name in df_toll.columns])

In [ ]:
df_toll.columns

['EXTRACTIONDATE', 'REG', '"Descripción series"', 'YEAR', 'DATE', 'INDEX']

In [ ]:
filtered_df_toll = df_toll.filter(df_toll['Descripción series'] == "Pasada de vehículos por plazas de peajes y pórticos de autopistas interurbanas, unidades")

In [ ]:
filtered_df_toll.show()

--------------------------------------------------------------------------------------------------------------------------
|"EXTRACTIONDATE"  |"REG"  |"Descripción series"                                |"YEAR"  |"DATE"      |"INDEX"           |
--------------------------------------------------------------------------------------------------------------------------
|2024-03-23        |15.0   |Pasada de vehículos por plazas de peajes y pórt...  |2014.0  |2014-12-31  |9557302.83333333  |
|2024-03-23        |15.0   |Pasada de vehículos por plazas de peajes y pórt...  |2015.0  |2015-12-31  |10483251.6666667  |
|2024-03-23        |15.0   |Pasada de vehículos por plazas de peajes y pórt...  |2016.0  |2016-12-31  |11826733.6666667  |
|2024-03-23        |15.0   |Pasada de vehículos por plazas de peajes y pórt...  |2017.0  |2017-12-31  |13521491.75       |
|2024-03-23        |15.0   |Pasada de vehículos por plazas de peajes y pórt...  |2018.0  |2018-12-31  |17355750.5        |
|2024-03-23     

In [ ]:
# Extract year and month for 'FechaEmision' in df_v
df_v = df_v.withColumn('Year', year('FECHAEMISION'))
df_v = df_v.withColumn('Month', month('FECHAEMISION'))

# Extract year and month for 'DATE' in df_debt
filtered_df_toll = filtered_df_toll.withColumn('Year', year('DATE'))
filtered_df_toll = filtered_df_toll.withColumn('Month', month('DATE'))

# Join the dataframes on year and month
df_with_toll= df_v.join(filtered_df_toll, (df_v['Year'] == filtered_df_toll['YEAR']))

# Select the desired columns including 'Debt(%GDP)' from df_debt
# Add any other columns you need from df_v and df_debt
df_with_toll = df_with_toll.select(df_v['*'], filtered_df_toll['INDEX'])

In [ ]:
# Upload as table to perform linear regression.
# Define the name of the table you want to create or overwrite in Snowflake
table_name = 'table_with_toll'

# Write the DataFrame to a new table in Snowflake
df_with_toll.write.mode('overwrite').save_as_table(table_name)

In [ ]:
tb = session.sql('select * from TABLE_WITH_TOLL ')
print(tb.columns)

['FECHAEMISION', 'DIASCOMPENSACION', '"l_kplx_YEAR"', '"l_kplx_MONTH"', 'INDEX']


In [ ]:
# Perform the linear regression
regression_query = """
SELECT
    REGR_SLOPE("DIASCOMPENSACION", "INDEX") AS slope,
    REGR_INTERCEPT("DIASCOMPENSACION", "INDEX") AS intercept,
    REGR_R2("DIASCOMPENSACION", "INDEX") AS r_squared,
    REGR_COUNT("DIASCOMPENSACION", "INDEX") AS sample_size,
    REGR_SXX("DIASCOMPENSACION", "INDEX") AS sxx,
    REGR_SYY("DIASCOMPENSACION", "INDEX") AS syy,
    REGR_SXY("DIASCOMPENSACION", "INDEX") AS sxy
FROM
    table_with_toll;
"""

# Execute the query
try:
    regression_results = session.sql(regression_query).collect()
    # Print the results
    for result in regression_results:
        print(f"Slope: {result['SLOPE']}")
        print(f"Intercept: {result['INTERCEPT']}")
        print(f"R-squared: {result['R_SQUARED']}")
        print(f"Sample size: {result['SAMPLE_SIZE']}")
        print(f"SXX: {result['SXX']}")
        print(f"SYY: {result['SYY']}")
        print(f"SXY: {result['SXY']}")
except SnowparkSQLException as e:
    print(f"An error occurred: {e}")

Slope: -6.874631498078042e-06
Intercept: 211.62254541643597
R-squared: 0.01876056098419247
Sample size: 334035865
SXX: 8.791411260150352e+21
SYY: 22146832611918.71
SXY: -6.043771276158765e+16


limited dataset - with p-value

limit 10000 observations in each year

In [ ]:
df_v=session.sql('select * from CONSOLIDATED_COPY')

In [ ]:
df_v = df_v.select([F.col(col_name).alias(col_name.replace('"', '')) for col_name in df_v.columns])

In [ ]:
# Step 1: Identify unique years
df_v_with_year = df_v.with_column("YEAR", year(col("FECHAEMISION")))
unique_years = df_v_with_year.select("YEAR").distinct().collect()

# Step 2: Iterate and filter for each year
filtered_dfs = []
for year_row in unique_years:
    year_value = year_row["YEAR"]
    df_filtered = df_v_with_year.filter(col("YEAR") == year_value).limit(10000)
    filtered_dfs.append(df_filtered)

# Step 3: Union results
final_df = filtered_dfs[0]
for df in filtered_dfs[1:]:
    final_df = final_df.union(df)

In [ ]:
df_toll = session.sql('select * from TRAFFIC_SANTIAGO_COPY')

In [ ]:
df_toll = df_toll.select([F.col(col_name).alias(col_name.replace('"', '')) for col_name in df_toll.columns])

In [ ]:
filtered_df_toll = df_toll.filter(df_toll['Descripción series'] == "Pasada de vehículos por plazas de peajes y pórticos de autopistas interurbanas, unidades")

In [ ]:
filtered_df_toll.show()

--------------------------------------------------------------------------------------------------------------------------
|"EXTRACTIONDATE"  |"REG"  |"Descripción series"                                |"YEAR"  |"DATE"      |"INDEX"           |
--------------------------------------------------------------------------------------------------------------------------
|2024-03-23        |15.0   |Pasada de vehículos por plazas de peajes y pórt...  |2014.0  |2014-12-31  |9557302.83333333  |
|2024-03-23        |15.0   |Pasada de vehículos por plazas de peajes y pórt...  |2015.0  |2015-12-31  |10483251.6666667  |
|2024-03-23        |15.0   |Pasada de vehículos por plazas de peajes y pórt...  |2016.0  |2016-12-31  |11826733.6666667  |
|2024-03-23        |15.0   |Pasada de vehículos por plazas de peajes y pórt...  |2017.0  |2017-12-31  |13521491.75       |
|2024-03-23        |15.0   |Pasada de vehículos por plazas de peajes y pórt...  |2018.0  |2018-12-31  |17355750.5        |
|2024-03-23     

In [ ]:
filtered_df_toll = filtered_df_toll.to_pandas()
final_df=final_df.to_pandas()

In [ ]:
#Convert 'FechaEmision' and 'DATE' to datetime
final_df['FECHAEMISION'] = pd.to_datetime(final_df['FECHAEMISION'])
filtered_df_toll['DATE'] = pd.to_datetime(filtered_df_toll['DATE'])

# Extract the year from 'FechaEmision' and 'Year'
final_df['YEAR'] = final_df['FECHAEMISION'].dt.year
filtered_df_toll['YEAR'] = filtered_df_toll['YEAR'].astype(int)

# Merge the dataframes on the 'Year' column
df_with_toll = pd.merge(final_df, filtered_df_toll[['YEAR', 'INDEX']], on='YEAR', how='inner')

# Drop the extra 'Year' column if you don't need it
df_with_toll.drop('YEAR', axis=1, inplace=True)

# Check the result
df_with_toll

,FECHAEXTRACCION,NUMERODOCUMENTO,NUMEROFOLIO,FECHAEMISION,FECHAVENCIMIENTO,FECHAREGISTRO,BUSINESSPARTNERID,CUENTACONTRATO,CLAVERECONCILIACION,CABECERA_FECHAHORACREACION,...,TRAMOMORA,INHABILITADO,CONCESIONARIA,TIPOCLIENTE,TIPOOPERACION,TIPODOCUMENTO,TIPOPRODUCTO,STATUSCOMPENSACION,STATUSCOMPENSACIONTMP,INDEX
0,2024-01-15 13:52:32.627,5.440048e+10,9367034.0,2017-04-26,2017-05-11,2017-04-26,0012810227,001102157946,17115FA060AF,2021-04-25 00:09:47,...,Pago,Cliente Tag,AVN,Natural,Peaje,Boleta,Tag,Pagada,Pagada,1.352149e+07
1,2024-01-15 13:52:32.627,5.440048e+10,9417020.0,2017-05-05,2017-05-22,2017-05-04,0011326883,001101499953,17123FA010CH,2021-04-25 00:17:20,...,Pago,Cliente Tag,AVN,Natural,Intereses,Boleta,Tag,Pagada,Pagada,1.352149e+07
2,2024-01-15 13:52:32.627,5.440048e+10,9448562.0,2017-05-09,2017-05-24,2017-05-09,0010647113,001100402788,17128FA020AX,2021-04-25 00:21:55,...,Pago,Cliente Tag,TSC,Natural,Peaje,Boleta,Tag,Pagada,Pagada,1.352149e+07
3,2024-01-15 13:52:32.627,5.440048e+10,3848977.0,2017-05-09,2017-05-24,2017-05-09,0010510108,001100265706,17128FA020AX,2021-04-25 00:21:55,...,Pago,Cliente Tag,AVN,Juridico,Peaje,Factura Electronica,Tag,Pagada,Pagada,1.352149e+07
4,2024-01-15 13:52:32.627,5.440048e+10,3848977.0,2017-05-09,2017-05-24,2017-05-09,0010510108,001100265706,17128FA020AX,2021-04-25 00:21:55,...,Pago,Cliente Tag,AVN,Juridico,Peaje,Factura Electronica,Tag,Pagada,Pagada,1.352149e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,2024-01-15 13:52:32.627,6.000135e+11,6639230.0,2023-07-04,2023-07-19,2023-07-04,0012102059,001101548118,230704R4/002,2023-07-05 07:12:29,...,[181 a 360] Dias,Cliente Tag,AVN,Juridico,Intereses,Factura Electronica,Tag,Impaga,Impaga,2.450204e+07
99996,2024-01-15 13:52:32.627,5.395134e+10,32368880.0,2023-09-26,2023-10-11,2023-09-26,0013195952,001102493788,23268FA060BD,2023-09-27 06:13:51,...,[91 a 180] Dias,Cliente Tag,AVN,Natural,Peaje,Boleta,Tag,Impaga,Impaga,2.450204e+07
99997,2024-01-15 13:52:32.627,1.760240e+10,31236765.0,2023-05-16,2023-05-31,2023-05-17,0012850656,001102193774,23136FA040BC,2023-05-18 07:06:02,...,Pago,Cliente Tag,AVN,Natural,Peaje,Boleta,Tag,Pagada,Pagada,2.450204e+07
99998,2024-01-15 13:52:32.627,5.605118e+10,29561927.0,2023-01-04,2023-01-19,2023-01-04,0012729528,001102084043,23003FA010AD,2023-01-05 06:40:43,...,[361+] Dias,Cliente Tag,AVN,Natural,Peaje,Boleta,Tag,Impaga,Impaga,2.450204e+07


In [ ]:
df_filtered = df_with_toll.dropna(subset=['INDEX', 'DIASCOMPENSACION'])

# Calculate correlation
correlation = df_filtered['INDEX'].corr(df_filtered['DIASCOMPENSACION'])

# Use statsmodels library for detailed statistics
X = sm.add_constant(df_filtered['INDEX'])  # Adding a constant term to the predictor
y = df_filtered['DIASCOMPENSACION']

model = sm.OLS(y, X)
results = model.fit()

# Print summary
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:       DIASCOMPENSACION   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     972.1
Date:                Tue, 26 Mar 2024   Prob (F-statistic):          2.64e-212
Time:                        21:36:27   Log-Likelihood:            -6.5075e+05
No. Observations:               92651   AIC:                         1.301e+06
Df Residuals:                   92649   BIC:                         1.302e+06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        179.0105      2.888     61.978      0.0

In [ ]:
print("Model Coefficients:")
print(f"Intercept: {results.params[0]:.3f}")
print(f"Slope (DIASCOMPENSACION): {results.params[1]:.3f}")

print("\nModel P-values:")
print(f"P-value (Intercept): {results.pvalues[0]:.3f}")
print(f"P-value (DIASCOMPENSACION): {results.pvalues[1]:.3f}")

print("\nModel R-squared: {:.3f}".format(results.rsquared))
print("Model Standard Errors:")
print(f"Standard Error (Intercept): {results.bse[0]:.3f}")
print(f"Standard Error (DIASCOMPENSACION): {results.bse[1]:.3f}")

Model Coefficients:
Intercept: 179.010
Slope (DIASCOMPENSACION): -0.000

Model P-values:
P-value (Intercept): 0.000
P-value (DIASCOMPENSACION): 0.000

Model R-squared: 0.010
Model Standard Errors:
Standard Error (Intercept): 2.888
Standard Error (DIASCOMPENSACION): 0.000


In [ ]:
session.close

<bound method Session.close of <snowflake.snowpark.session.Session object at 0x7cd8eb01b2e0>>